In [ ]:
pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


In [2]:
df = pd.read_csv('C:/Users/2024/ORC/MLOp/Housing_Prediction/Training.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
import pandas as pd
def read_dataframe(filename):
    df = pd.read_csv(filename)
    df = df.drop('Id', axis=1)
    df = df.drop(columns = ['Alley', 'PoolQC', 'Fence', 'MiscFeature'])

    df = df[df.Neighborhood == 'Avondale']
    df = df[(df.SalePrice >= 100000) & (df.SalePrice <= 500000)]

    categorical = ['BedroomAbvGr', 'GarageCars']
    df[categorical].astype(str)

    return df

# df[categorical].astype(str).dtypes


In [5]:
df_train = read_dataframe('C:/Users/2024/ORC/MLOp/Housing_Prediction/Training.csv')
df_val = read_dataframe('C:/Users/2024/ORC/MLOp/Housing_Prediction/Validation.csv')

In [6]:
len(df_train), len(df_val)

(288, 49)

In [7]:
df_train['BedroomAbvGr'] = df_train['BedroomAbvGr'].astype(str)
df_train['GarageCars'] = df_train['GarageCars'].astype(str)

df_train['BED_GAR'] = df_train['BedroomAbvGr'] + '_' + df_train['GarageCars']

df_val['BedroomAbvGr'] = df_val['BedroomAbvGr'].astype(str)
df_val['GarageCars'] = df_val['GarageCars'].astype(str)

df_val['BED_GAR'] = df_val['BedroomAbvGr'] + '_' + df_val['GarageCars']


In [ ]:
df_val

In [9]:
categorical = ['BED_GAR']#'BedroomAbvGr', 'GarageCars']
numerical = ['GarageArea']

dv = DictVectorizer()

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [10]:
target = 'SalePrice'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

C:\Users\2024\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


36147.19539648093

In [12]:
la = Lasso(alpha=10000000)
la.fit(X_train, y_train)

y_pred = la.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

C:\Users\2024\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


25833.66489247413

In [13]:
if not os.path.exists('models'):
    os.makedirs('models')

# original code to save the file
with open('models/lasso.bin', 'wb') as f_out:
    pickle.dump((dv, la), f_out)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')
plt.legend()